# Introduction to Large Language Model Hosting on SageMaker with DeepSpeed Container


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

---


In this notebook, we explore how to host a large language model on SageMaker using the [Large Model Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html) container that is optimized for hosting large models using DJLServing. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

Language models have recently exploded in both size and popularity. In 2018, BERT-large entered the scene and, with its 340M parameters and novel transformer architecture, set the standard on NLP task accuracy. Within just a few years, state-of-the-art NLP model size has grown by more than 500x with models such as OpenAI’s 175 billion parameter GPT-3 and similarly sized open source Bloom 176B raising the bar on NLP accuracy. This increase in the number of parameters is driven by the simple and empirically-demonstrated positive relationship between model size and accuracy: more is better. With easy access from models zoos such as Hugging Face and improved accuracy in NLP tasks such as classification and text generation, practitioners are increasingly reaching for these large models. However, deploying them can be a challenge because of their size.

In this notebook, we deploy the open source GPT-J Model which is comprised of 6B parameters on a single GPU. Along the way we will explore approaches that will allow us to scale to larger models with practically no code changes.

This notebook was tested on a `ml.t3.medium` instance using the `Python 3 (Data Science)` kernel on SageMaker Studio.

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [ ]:
!pip install sagemaker boto3 --upgrade  --quiet

In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "large-model-djl-gptj6b/code"  # folder within bucket where code artifact will go
s3_model_prefix = "hf-large-model-djl-gptj6b/model"  # folder where model checkpoint will go
default_bucket_prefix = sess.default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    s3_code_prefix = f"{default_bucket_prefix}/{s3_code_prefix}"
    s3_model_prefix = f"{default_bucket_prefix}/{s3_model_prefix}"

region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment()  # jinja environment to generate model configuration templates

In [ ]:
# lookup the inference image uri based on our current region
inference_image_uri = sagemaker.image_uris.retrieve(
    "djl-deepspeed", region=region, version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

In [ ]:
# lookup the S3 model location based on our region
pretrained_model_location = f"s3://sagemaker-example-files-prod-{region}/models/gpt-j-6b-model/"
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

## Deploying a Large Language Model using Hugging Face Accelerate
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("accelerate_src/serving.template").open().read())
Path("accelerate_src/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize accelerate_src/serving.properties | cat -n

There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.
6. `option.load_in_8bit` - Quantizes the model weights to int8 thereby greatly reducing the memory footprint of the model from the initial FP32. See this [blog post](https://huggingface.co/blog/hf-bitsandbytes-integration) from Hugging Face for additional information 

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. These are exposed through the `device_map` and `load_in_8bit` parameters which enable sharding and shrinking of the model. The sharding approach taken here is layer wise as individual model layers are placed onto different GPU devices and data flows sequentially from the input to the final output layer as illustated below<br>
<img src="images/LayerShard.png" width="800"/>

Even though in this example the model will be running on a single GPU and will not be sharded, this parameter would automatically apply sharding as we scale to larger models on multi-GPU instances.

We place the `serving.properties` file into a tarball and upload it to S3

In [ ]:
!tar czvf acc_model.tar.gz accelerate_src/

In [ ]:
hf_s3_code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {hf_s3_code_artifact}")

## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [ ]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(image_uri=image_uri, model_data=model_data, role=role)

    model.deploy(initial_instance_count=1, instance_type=instance_type, endpoint_name=endpoint_name)

    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer(),
    )

    return predictor

In [ ]:
# creates a unique endpoint name
hf_endpoint_name = sagemaker.utils.name_from_base("gptj-acc")
print(f"Our endpoint will be called {hf_endpoint_name}")

In [ ]:
# deployment will take about 10 minutes
hf_predictor = deploy_model(
    image_uri=inference_image_uri,
    model_data=hf_s3_code_artifact,
    role=role,
    endpoint_name=hf_endpoint_name,
    instance_type="ml.g4dn.4xlarge",
    sagemaker_session=sess,
)

Let's run an example with a basic text generation prompt `Large model inference is`

In [ ]:
hf_predictor.predict(
    {"inputs": "Large model inference is", "parameters": {"max_length": 50, "temperature": 0.5}}
)

Now let's try another example where we provide the a few samples of text and sentiment pairs and ask it to classify a new example

In [ ]:
print(
    hf_predictor.predict(
        {
            "inputs": """Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment:""",
            "parameters": {"max_length": 50, "temperature": 0.5},
        }
    )[0]["generated_text"]
)

You can see that the model filled in a Sentiment value for the last example. You can take a look at a blog post [here](https://towardsdatascience.com/how-to-use-gpt-j-for-almost-any-nlp-task-cb3ca8ff5826) for more examples of prompts. Finally Let's do a quick benchmark to see what kind of latency we can expect from this model

In [ ]:
%%timeit -n3 -r1
hf_predictor.predict(
    {"inputs": "Large model inference is", "parameters": {"max_length": 50, "temperature": 0.5}}
)

In [ ]:
# Clean up the endpoint before proceeding
hf_predictor.delete_endpoint()

## Bonus: Deploying a Large Language Model Using DeepSpeed
Now we will explore another approach for deploying Large Language Models using [DeepSpeed](https://www.deepspeed.ai/). DeepSpeed provides various [inference optimizations](https://www.deepspeed.ai/tutorials/inference-tutorial/) for compatible transformer based models including model sharding, optimized inference kernels, and quantization. To leverage DeepSpeed, we simply need to modify our `serving.properties` file

In [ ]:
template = jinja_env.from_string(Path("deepspeed_src/serving.template").open().read())
Path("deepspeed_src/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize deepspeed_src/serving.properties | cat -n

Notice that the `engine` parameter is now set to `DeepSpeed` and the `option.entryPoint` has been modified to use the `deepspeed.py` module. Python scripts that use DeepSpeed can not be launched as traditional python scripts (i.e. python `deepspeed.py` would not work.) Setting `engine=DeepSpeed` will automatically configure the environment and launch the inference script appropriatelly.  
The only other new parameter here is `option.tensor_parallel_degree` where we have to specify the number of GPU devices to which the model will be sharded.

Unlike Accelerate where the model was partitioned along the layers, DeepSpeed uses TensorParallelism where individual layers (Tensors) are sharded accross devices. For example each GPU can have a slice of each layer. The diagram below provides a high level illustartion of how this works <br>

<img src="images/TensorShard.png" width="800"/>

Where with the layer-wise approach, the data fllowed through each GPU device sequeantially, here data is sent to all GPU devices where a partial result is compute on each GPU. The partial results are then collected though an All-Gather operation to compute the final result. 
TensorParallelism generally provides higher GPU utilization and better performance.

In [ ]:
!tar czvf ds_model.tar.gz deepspeed_src/

In [ ]:
ds_s3_code_artifact = sess.upload_data("ds_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {ds_s3_code_artifact}")

In [ ]:
ds_endpoint_name = sagemaker.utils.name_from_base("gptj-ds")
ds_predictor = deploy_model(
    image_uri=inference_image_uri,
    model_data=ds_s3_code_artifact,
    role=role,
    endpoint_name=ds_endpoint_name,
    instance_type="ml.g4dn.4xlarge",
    sagemaker_session=sess,
)

In [ ]:
ds_predictor.predict(
    {"inputs": "Large model inference is", "parameters": {"max_length": 50, "temperature": 0.5}}
)

In [ ]:
print(
    ds_predictor.predict(
        {
            "inputs": """Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment:""",
            "parameters": {"max_length": 50, "temperature": 0.5},
        }
    )[0]["generated_text"]
)

In [ ]:
%%timeit -n3 -r1
ds_predictor.predict(
    {"inputs": "Large model inference is", "parameters": {"max_length": 50, "temperature": 0.5}}
)

In [ ]:
ds_predictor.delete_endpoint()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|llm-workshop|lab1-deploy-llm|intro_to_llm_deployment.ipynb)
